# Resolved-rate control of the end-effector

The goal of this and the next laboratory is to implement a resolved-rate controller to control the end-effector. You should aim to at least finish Part 1 during Lab 5 and Part 2 during Lab 6. We recommend that you read all the instructions in Part 2 well before Lab 6.

Last laboratory, we used analytic IK to compute desired positions for the joint angles that would ensure we reach a desired position (touching the pink balls). Then we generated a straight line trajectory in joint space and used the joint PD controller to track this trajectory.

Here, we will use the fact that $$\mathcal{V}_{SO}^O = J_O(\boldsymbol{\theta}) \cdot \dot{\boldsymbol{\theta}}$$ 
to derive directly a controller acting in the end-effector space,
where $\boldsymbol{\theta} = [\theta_0, \theta_1, \theta_2]$ is the vector containing all the joint positions and $J_O$ is the Jacobian used to compute twists in Frame 0.

## Instructions
Answer all the questions below and submit a pdf with detailed answers to these questions, including the plots through Brightspace. You will also need to submit the Jupyter notebooks with the code used to answer the questions.

## Robot model
Consider the following kinematic model of the robot <img src="./3d_robot_model.png" width="700">


In [1]:
#setup nice plotting
%matplotlib notebook

# we import useful libraries
import time
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt

# we import the robot class
use_real_robot = False

if use_real_robot:
    from nyu_finger import NYUFingerReal    
else:
    from nyu_finger_simulator import NYUFingerSimulator
    
    
# here we define the global variables for the robot size
l0 = 0.3
l1 = 0.16
l2 = 0.16
l3 = 0.014

pybullet build time: Aug 21 2022 18:46:39


In [2]:
# Import needed previously defined functions
def translate(vector): # vector: np.array([1,2,3])
    transform = np.eye(4)
    transform[:3,3:] = vector.reshape(3,1)
    return transform


def rotateX(angle):
    transform = np.eye(4)
    transform[1:3, 1:3] = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
    return transform


def rotateY(angle):
    transform = np.eye(4)
    transform[:3, :3] = np.array([[np.cos(angle), 0, np.sin(angle)], [0,1,0], [-np.sin(angle), 0, np.cos(angle)]])
    return transform


def rotateZ(angle):
    transform = np.eye(4)
    transform[:2, :2] = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
    return transform


def forward_kinematics(theta):
    # assume theta is a vector, theta[0] = theta0, theta[1] = theta1, etc.
    # we define the lengths of the robot
#     l0 = 0.3
#     l1 = 0.16
#     l2 = 0.16
#     l3 = 0.014

    T_SH1 = translate(np.array([l0, 0, 0]))
    T_H1H2 = rotateX(theta[0]) @ translate(np.array([0, 0, l3])) @ rotateZ(theta[1])
    T_H2K = translate(np.array([0, -l1, 0])) @ rotateZ(theta[2])
    T_KF = translate(np.array([0, -l2, 0]))
    
    T_SF = T_SH1 @ T_H1H2 @ T_H2K @ T_KF
    
    return T_SF


def adjoint(T):
    ''' Gives the adjoint(6x6) of a transformation matrix'''
    AdT = np.zeros((6,6))
    R = T[:3, :3]
    p = T[:3, 3]
    AdT[:3, :3] = R
    AdT[3:6, 3:6] = R
    AdT[3:6, :3] = vec2skewsym(p) @ R
    return AdT


def vec2skewsym(array):
    ''' Bracket notation'''
    a_skew = np.zeros((3,3))
    a_skew[2][1], a_skew[1][2] = array[0], -array[0]
    a_skew[0][2], a_skew[2][0] = array[1], -array[1]
    a_skew[1][0], a_skew[0][1] = array[2], -array[2]
    return a_skew


def get_only_IK(x,y):
    '''
    Input: (x,y) position of the fingertip
    Output: one of theta answers (max 1,3)
    '''
    # Use the first solution, account for no solution
    q_theta = inverse_kinematics(x,y)
    if len(q_theta) != 0:
        return np.array(q_theta[0])
    
    
def inverse_kinematics(x,y):
    """
    inverse kinematics function
    input (x,y) position of the fingertip
    output a list of 3D vectors which are possible solutions to the problem 
    (the list is empty if there are no solutions)
    """
    sol = []
    l_sq = (x - l0) ** 2 + y ** 2
    l_max_sq = (l1 + l2) ** 2
    beta = np.arctan2(y,x-l0)
    theta1 = np.pi/2 + beta
    
    if l_sq > l_max_sq:
        pass
    
    elif l_sq == l_max_sq:
        sol.append([0, theta1, 0])
        
    else:
        alpha = np.arccos((l_sq + l1**2 - l2**2) / (2 * l1 * np.sqrt(l_sq)))
        theta1_p = theta1 - alpha
        theta2_p = np.arccos((l_sq - l1**2 - l2**2) / (2 * l1 * l2))
        sol.append([0, theta1_p, theta2_p])
        
        theta1_n = theta1 + alpha
        theta2_n = - theta2_p
        sol.append([0, theta1_n, theta2_n])
    
    return sol
    # return a list of solutions [[theta0, theta1, theta2]]
    
    
def compute_trajectory(position_init, position_goal, t_init, t_goal, t):
    ''' Parameter can be angle or coordinates, 5th order'''
    T = t_goal - t_init
    dt = t - t_init
    dpos = position_goal - position_init
    pos_param = 10 * (dt / T) ** 3 - 15 * (dt / T) ** 4 + 6 * (dt / T) ** 5
    vel_param = (30 * (dt / T) ** 2 - 60 * (dt / T) ** 3 + 30 * (dt / T) ** 4) / T
    desired_position = position_init + pos_param * dpos
    desired_velocity = vel_param * dpos
    
    # we return the answer
    return desired_position, desired_velocity

# Part 1: compute the end-effector Jacobian

1. Write a ``compute_jacobian`` function which computes the Jacobian of the end-effector. The computed Jacobian should be with respect to a frame located at the end-effector but oriented like the spatial frame (Frame O in the figure above). You will need to reuse the kinematics functions written in the previous laboratories.

Here make sure to return a 6x3 matrix (you will take the rows corresponding to the linear velocity in the following questions).

2. To test your Jacobian function, use your code from Lab 4 that makes the robot reach 3 different goals and compute the velocity of the end-effector at each instant of time. Compare the computed velocities with the velocities you get by numerically differentiating the end-effector positions (i.e. by computing $v[t] = \frac{x[t] - x[t-1]}{\Delta t}$). 

Here you will ignore the angular velocity and only consider the linear velocity of the end-effector. If we only look at the linear velocity of point O (i.e. the velocity of the end-effector with respect to frame O), we can decompose the Jacobian into two part (the top 3 rows are associated to the angular velocity of the end-effector and the bottom 3 are associated to the linear velocity): $$J_O(\boldsymbol{\theta}) = \begin{bmatrix} J_{O\omega} \\ J_{Ov} \end{bmatrix}$$
and we can write the velocity equation as
$$\mathcal{V}_{SO}^O = \begin{pmatrix} \omega_O \\ v_O \end{pmatrix} = \begin{bmatrix} J_{O\omega} \\ J_{Ov} \end{bmatrix} \cdot \dot{\boldsymbol{\theta}}$$

So the linear velocity of the end-effector is $$v_O = J_{Ov} \dot{\boldsymbol{\theta}}$$


3. Plot the velocities computed in question 2.

In [3]:
def compute_jacobian(theta):
    # First, we can calculate Jacobian in spacial frame
    T_SH1 = translate(np.array([l0, 0, 0]))
    T_H1H2 = rotateX(theta[0]) @ translate(np.array([0, 0, l3])) @ rotateZ(theta[1])
    T_H2K = translate(np.array([0, -l1, 0])) @ rotateZ(theta[2])
    T_KF = translate(np.array([0, -l2, 0]))
    

    # Getting Jacobian of each joint and stack horizontally
    w_H1 = np.array([[1,0,0,0,0,0]])
    J1_S = adjoint(T_SH1) @ w_H1.T

    w_H2 = np.array([[0,0,1,0,0,0]])
    J2_S = adjoint(T_SH1 @ T_H1H2) @ w_H2.T
 
    w_K = np.array([[0,0,1,0,0,0]])
    J3_S = adjoint(T_SH1 @ T_H1H2 @ T_H2K) @ w_K.T
    
    J_S = np.hstack([J1_S, J2_S, J3_S])
    
    
    # Get transformation matrix of O in spacial frame
    T_SO = forward_kinematics(theta)
    T_SO[:3, :3] = np.zeros(3)
    T_OS = np.eye(4)
    T_OS[:3, 3] = -T_SO[:3, 3]

    
    # Convert from J_S to J_O
    J_O = np.zeros([3,3])
    J_O = adjoint(T_OS) @ J_S
    
    return J_O


# test
compute_jacobian([0,0,0])

array([[ 1.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ],
       [ 0.   ,  1.   ,  1.   ],
       [ 0.   ,  0.32 ,  0.16 ],
       [-0.014,  0.   ,  0.   ],
       [-0.32 ,  0.   ,  0.   ]])

In [4]:
if use_real_robot:
    # Triggers the real robot
    # Don't forget to turn on the robot first !!!
    robot = NYUFingerReal()
    # Make sure that the motor number matches that on the robot
    motor_number = np.array([3,2,0])
    robot.initialize('ens1', motor_number)
else:
    # we can now create a robot simulation
    robot = NYUFingerSimulator()

# we reset the simulation
if not use_real_robot:
    robot.reset_state([0,0,0])

    
# we simulate for 15 seconds (5 seconds per goal)
run_time = 15.
dt = 0.001
num_steps = int(run_time/dt)

# the PD gains
P = np.array([4,4,4])
D = np.array([0.05, 0.05, 0.05])

## we allocate a few vectors to collect data while we do the simulation
# we will store the sensor values (angular position and velocities)
measured_positions = np.zeros([num_steps,3])
measured_velocities = np.zeros_like(measured_positions)
# we will store the commands sent to the robot
desired_torques = np.zeros_like(measured_positions)
# we will store the desired joint trajectories that the PD controller follows
desired_positions = np.zeros_like(measured_positions)
desired_velocities = np.zeros_like(measured_positions)
# we keep track of time
time = np.zeros([num_steps])
# we will store the end effector position (desired and measured)
measured_finger_position = np.zeros([num_steps,3])
desired_finger_position = np.zeros([num_steps,3])

measured_linear_velocity = np.zeros([num_steps,3])
computed_linear_velocity = np.zeros([num_steps,3])

# here we create a list of ball positions and display them
ball_positions = [np.array([0.597,-0.056]), np.array([0.521,0.12]), np.array([0.3,-0.225])]
if not use_real_robot:
    for ball in ball_positions:
        robot.add_ball(ball[0], ball[1])
else:
    # we wait
    for i in range(7500):
        robot.send_joint_torque(np.zeros([3,1]))
        robot.step()

# the time to reach a goal
time_to_goal = 5.0

# we use this to know which target we are currently aiming
ball_number = 0

# we measure the internal time for one target movement
t = 0.

'''Initialize all goals'''
start_pos = np.array([l0, -(l1 + l2)])
goals = [start_pos] + ball_positions


'''Lab5'''

# Select either fifth interpolation or linear interpolation
linear_interp = False

for i in range(num_steps):
    # get the current time and save it
    time[i] = dt * i
    
    # we read the position and velocities of the joints from the robot or simulation
    q, dq = robot.get_state()
    
    # we store these values for later use
    measured_positions[i,:] = q      # angle
    measured_velocities[i,:] = dq    # angle
    
    # save the current position of the fingertip using the FK function
    pose = forward_kinematics(q)
    measured_finger_position[i,:] = pose[0:3,3]    # coord
    
    
    '''Key code'''
    measured_linear_velocity[i,:] = (measured_finger_position[i,:] - measured_finger_position[i-1,:])/dt
    computed_linear_velocity[i,:] = compute_jacobian(q)[3:,:] @ dq.T

    
    '''Control code starts'''
    
    # Identify the start and end time for each path
    cur_path = int(time[i] // time_to_goal)
    t_init = cur_path * time_to_goal
    t_goal = (cur_path + 1) * time_to_goal
    
    # Identify the start and end coordinates/angles for each path
    start_coord = goals[cur_path]
    end_coord = goals[cur_path+1]
    start_theta = get_only_IK(start_coord[0], start_coord[1])
    end_theta = get_only_IK(end_coord[0], end_coord[1])
    
    if linear_interp:
        dq_des = (end_theta - start_theta) / time_to_goal
        q_des = start_theta + (time[i] - t_init) * dq_des
    else:
        q_des, dq_des = compute_trajectory(start_theta, end_theta, t_init, t_goal, time[i])
        
    '''Control code ends'''
    
    # we save the desired positions/velocities for later plotting 
    desired_positions[i,:] = q_des
    desired_velocities[i,:] = dq_des
    
    ##PD controller
    error = q_des - q # the position error for all the joints (it's a 3D vector)
    d_error = dq_des-dq # the velocity error for all the joints
    
    # we compute the desired torques as a PD controller
    joint_torques = P * error + D * d_error
    desired_torques[i,:] = joint_torques
    
    # we send them to the robot and do one simulation step
    robot.send_joint_torque(joint_torques)
    robot.step()

In [5]:
def plot_finger_velocity(measured_linear_velocity, computed_linear_velocity):
    fig = plt.figure()
  
    plt.subplot(3,1,1)
    plt.plot(time[1:], measured_linear_velocity[1:,0], 'b-', time[1:], computed_linear_velocity[1:,0], '--k')
    plt.ylabel(r'$x$')
    
    plt.subplot(3,1,2)
    plt.plot(time[1:], measured_linear_velocity[1:,1], 'b-', time[1:], computed_linear_velocity[1:,1], '--k')
    plt.ylabel(r'$y$')
    
    plt.subplot(3,1,3)
    plt.plot(time[1:], measured_linear_velocity[1:,2], 'b-', time[1:], computed_linear_velocity[1:,2], '--k')
    plt.ylabel(r'$z$')
    plt.xlabel('Time [s]')
    
    
plot_finger_velocity(measured_linear_velocity, computed_linear_velocity)

<IPython.core.display.Javascript object>